In [28]:
# -*- coding: utf-8 -*-
import cv2
import numpy as np
import pandas as pd 
from read_path import *
import re


In [2]:
def     read_imgpath(data_folder):
    names = get_list_filename(data_folder)
    labels = [re.split('_',name)[0] for name in names]
    data_df = pd.DataFrame({'img_name':names, 'label':labels})
    return data_df


data_folder = '../../example/scence_recognition_small'
data_df = read_imgpath(data_folder)
print data_df
    

train, test = data_df[:int (0.8 * len(data_df))], data_df[int(0.8 * len(data_df)) : -1]
print len(train), len(test)

                       img_name         label
0            forest_sclos13.jpg        forest
1           highway_bost150.jpg       highway
2           highway_natu778.jpg       highway
3           mountain_n18019.jpg      mountain
4            street_a205062.jpg        street
5            forest_cdmc271.jpg        forest
6      tallbuilding_urban15.jpg  tallbuilding
7                coast_bea2.jpg         coast
8              street_urb19.jpg        street
9         insidecity_city84.jpg    insidecity
10      opencountry_sopen11.jpg   opencountry
11           mountain_ski61.jpg      mountain
12          insidecity_gre9.jpg    insidecity
13         insidecity_hous1.jpg    insidecity
14      insidecity_boston29.jpg    insidecity
15         insidecity_par40.jpg    insidecity
16         opencountry_nat8.jpg   opencountry
17             street_enc48.jpg        street
18          mountain_moun10.jpg      mountain
19             coast_osun54.jpg         coast
20   tallbuilding_boston210.jpg  t

In [11]:
# from style_feature import Style_Feature
import numpy as np
from feature_extraction.cielab import cielab
from feature_extraction.lmgist import lmgist, param_gist

def style_feature_extraction(img, lab = True, gist = True, gist_parameters = None):
    if gist and gist_parameters is None:
        param = param_gist()
        param.img_size = 256
        param.orientations_per_scale = [8, 8, 8, 8]
        param.number_blocks = 4
        param.fc_prefilt = 4        
    feature = None
    if lab:
        feature = cielab(img).astype(float)
    if gist:
        if feature is None:
            feature = lmgist(img, param)[0].astype(float)
        else:
            feature = np.concatenate((feature, lmgist(img, param)[0].astype(float)))
    return feature

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

img = np.array(mpimg.imread('../../example/1.jpg'))
style_feature = style_feature_extraction(img)
print style_feature.shape

(1296,)


In [13]:
train

,img_name,label
0,forest_sclos13.jpg,forest
1,highway_bost150.jpg,highway
2,highway_natu778.jpg,highway
3,mountain_n18019.jpg,mountain
4,street_a205062.jpg,street
5,forest_cdmc271.jpg,forest
6,tallbuilding_urban15.jpg,tallbuilding
7,coast_bea2.jpg,coast
8,street_urb19.jpg,street
9,insidecity_city84.jpg,insidecity


In [19]:
img_paths = [os.path.join(data_folder,img_name) for img_name in train['img_name']]
features_train = np.array([style_feature_extraction(load_img(img_path)) for img_path in img_paths])

In [26]:
len(train['label'])

255

In [37]:
import sklearn

def svm(X, y, C=1.0, class_weight=None, dual=True, fit_intercept=True, \
        intercept_scaling=1, loss='squared_hinge', max_iter=1000, \
        multi_class='ovr', penalty='l2', random_state=None, tol=0.0001, verbose=0):
    clf = sklearn.svm.LinearSVC(C=C, class_weight=class_weight, dual=dual, fit_intercept=fit_intercept, \
        intercept_scaling=intercept_scaling, loss=loss, max_iter=max_iter, \
        multi_class=multi_class,penalty=penalty, random_state=random_state, tol=tol, verbose=verbose)
    clf.fit(X, y)
    return clf
    
clf = svm(features_train, train['label'])

In [51]:
print clf.coef_.shape, clf.intercept_.shape

(8, 1296) (8,)


In [55]:
pred_train = clf.predict(features_train)
print 'Accuracy: ', np.sum(pred_train == train['label']) * 1.0/len(pred_train)


Accuracy:  0.450980392157


In [53]:
img_paths = [os.path.join(data_folder,img_name) for img_name in test['img_name']]
features_test = np.array([style_feature_extraction(load_img(img_path)) for img_path in img_paths])

In [56]:
pred_test = clf.predict(features_test)
print 'Accuracy: ', np.sum(pred_test == test['label']) * 1.0/len(pred_test)


Accuracy:  0.444444444444


In [ ]:
sklearn.svm